<a href="https://colab.research.google.com/github/robertritz/Mongolian-Data-Stories/blob/master/MNT%20Exchange%20Rate%20Forecast/Part%202%20-%20Machine%20Learning%20Model/AutoML_with_TPOT_for_Exchange_Rate_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoML with TPOT for Mongolian Exchange Rate Forecasting

### Install TPOT and dependencies

In [1]:
!pip install deap update_checker tqdm stopit

    100% |████████████████████████████████| 942kB 6.3MB/s 
  Running setup.py bdist_wheel for deap ... - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/22/ea/bf/dc7c8a2262025a0ab5da9ef02282c198be88902791ca0c6658
  Running setup.py bdist_wheel for stopit ... - done
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built deap stopit


In [2]:
!pip install tpot

    100% |████████████████████████████████| 901kB 19.5MB/s 
  Running setup.py bdist_wheel for tpot ... - \ done
  Stored in directory: /root/.cache/pip/wheels/97/86/27/698a1366f854d2344fd0f7ee9ef57c8044ec02f182c8a0431e
Successfully built tpot


###Set up Google Drive FUSE

In [3]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22298 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu2~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu2~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu2~ubuntu18.04.1) ...


In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [5]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force

Enter verification code: ··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [6]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print('Files in Drive:')
!ls

Files in Drive:
adc.json  drive  sample_data


Change directory to project data directory

In [0]:
import os
os.chdir('drive/Data Science/Mongolian Data Stories/Github Repository/MNT Exchange Rate Forecast/Part 2 - Machine Learning Model/Data/Modified Data')

In [8]:
!ls

'Balance of Payments - Edited.csv'   data.csv  'Training-Test Data'


### Import data

Import X, y data and split to training and test data

In [0]:
import pandas as pd
import numpy as np

In [0]:
X = pd.read_csv('Training-Test Data/X.csv', index_col=0)
y = pd.read_csv('Training-Test Data/y.csv', index_col=0)

In [11]:
y

,y
0,1155.35
1,1150.91
2,1146.09
3,1144.37
4,1170.64
5,1267.51
6,1381.66
7,1470.17
8,1524.07
9,1423.86


In [0]:
#split into training and test data
X_train = X[:100].values
X_test = X[100:].values

y_train = y[:100].values
y_test = y[100:].values

In [15]:
y_test

array([[2466.64],
       [2489.53],
       [2459.44],
       [2470.83],
       [2447.45],
       [2419.42],
       [2397.97],
       [2349.87],
       [2444.95],
       [2430.05],
       [2464.43],
       [2456.17],
       [2443.57],
       [2427.13],
       [2417.71],
       [2394.23],
       [2392.77],
       [2402.86],
       [2409.8 ],
       [2462.82],
       [2463.25],
       [2472.99],
       [2552.13],
       [2564.44]])

## Implement TPOT

Our current predicted variable is for 3 months ahead of current. TPOT does not work easily with multi output regression, so we can break this into more reasonable chunks. 

Later we can do this again for 6 months and 12 months to create a model for those time frames.

In [0]:
from tpot import TPOTRegressor

In [0]:
pipeline_optimizer = TPOTRegressor(generations=5, population_size=20, max_time_mins=480, n_jobs=-1, verbosity=2, random_state=12345)
pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))
pipeline_optimizer.export('drive/Data Science/Mongolian Data Stories/Github Repository/MNT Exchange Rate Forecast/Part 2 - Machine Learning Model/Output/tpot_exported_pipeline.py')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Optimization Progress: 100%|██████████| 40/40 [00:08<00:00,  5.23pipeline/s]

Generation 1 - Current best internal CV score: -21448.612820112256


Optimization Progress: 100%|██████████| 60/60 [00:12<00:00,  6.70pipeline/s]

Generation 2 - Current best internal CV score: -21007.43215706372


Optimization Progress: 100%|██████████| 80/80 [00:16<00:00,  4.13pipeline/s]

...

Optimization Progress: 100%|██████████| 1200/1200 [11:54<00:00,  1.07pipeline/s]

Generation 59 - Current best internal CV score: -13739.864363072022


Optimization Progress: 100%|██████████| 1220/1220 [12:13<00:00,  1.35pipeline/s]

Generation 60 - Current best internal CV score: -13739.864363072022


Optimization Progress:  99%|█████████▉| 1229/1240 [12:22<00:13,  1.23s/pipeline]